# SERVER

In [8]:
%%writefile UDPs.py

##############################################################################
import pygame
import cv2
import random
import math
import numpy as np
# our files
import functions
import shapes


N = 4 # the number of players
FPS = 30 # frames per second
FALL_SPEED = 1.0 # basic fall speed
# video source selection
SOURCE = 'camera'
# SOURCE = 'video'
# cam_idx = 0
cam_idx = -1

layout = functions.get_layout(N) # select screen layout
pygame.font.init() # initialize font

# nearest integer
if N > 1:
    q = 2*math.ceil(N/2)
else:
    q = 1
cell = np.arange(q).reshape(layout[0],layout[1])

# GLOBALS VARS
if SOURCE == 'video':
    file_path = "v1.mp4"
    vid = cv2.VideoCapture(file_path)
    H,W = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
elif SOURCE == 'camera':
    W,H = 320,240
    # W,H = 160,120
s_width = W*layout[0]
s_height = H*layout[1]
block_size = 20

N_ver_blocks = s_height//block_size 
N_hor_blocks = s_width//block_size

play_width = N_hor_blocks*block_size  # meaning 300 // 10 = 30 width per block
play_height = N_ver_blocks*block_size  # meaning 600 // 20 = 30 height per block

top_left_x = (s_width - play_width) // 2
top_left_y = s_height - play_height

shapes,shape_colors = shapes.get_shapes_and_colors()

class Piece(object):
    def __init__(self, x, y, shape):
        self.x = x
        self.y = y
        self.shape = shape
        self.color = shape_colors[shapes.index(shape)]
        self.rotation = 0


def create_grid(locked_pos={}):
    grid = [[(0,0,0) for _ in range(N_hor_blocks)] for _ in range(N_ver_blocks)]
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if (j, i) in locked_pos:
                c = locked_pos[(j,i)]
                grid[i][j] = c
    return grid


def convert_shape_format(shape):
    positions = []
    format = shape.shape[shape.rotation % len(shape.shape)]

    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                positions.append((shape.x + j, shape.y + i))

    for i, pos in enumerate(positions):
        positions[i] = (pos[0] - 2, pos[1] - 4)
    return positions


def valid_space(shape, grid):
    accepted_pos = [[(j, i) for j in range(N_hor_blocks) if grid[i][j] == (0,0,0)] for i in range(N_ver_blocks)]
    accepted_pos = [j for sub in accepted_pos for j in sub]

    formatted = convert_shape_format(shape)

    for pos in formatted:
        if pos not in accepted_pos:
            if pos[1] > -1:
                return False
    return True


def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < 1:
            return True
    return False


def get_shape():
    return Piece(random.randint(3,N_hor_blocks-3), 0, random.choice(shapes))


def draw_text_middle(surface, text, size, color):
    font = pygame.font.SysFont("comicsans", size, bold=True)
    label = font.render(text, 1, color)
    surface.blit(label, (top_left_x + play_width /2 - (label.get_width()/2), top_left_y + play_height/2 - label.get_height()/2))


def draw_grid(surface, grid):
    sx = top_left_x
    sy = top_left_y
    for i in range(len(grid)):
        pygame.draw.line(surface, (128,128,128), (sx, sy + i*block_size), (sx+play_width, sy+ i*block_size))
        for j in range(len(grid[i])):
            pygame.draw.line(surface, (128, 128, 128), (sx + j*block_size, sy),(sx + j*block_size, sy + play_height))


def clear_rows(grid, locked):
    inc = 0
    for i in range(len(grid)-1, -1, -1):
        row = grid[i]
        if (0,0,0) not in row:
            inc += 1
            ind = i
            for j in range(len(row)):
                try:
                    del locked[(j,i)]
                except:
                    continue

    if inc > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                newKey = (x, y + inc)
                locked[newKey] = locked.pop(key)

    return inc

# win = pygame.display.set_mode((s_width, s_height))
##############################################################################

import cv2, imutils, socket
import numpy as np
import base64
import time

import os
from _thread import *

import argparse
parser = argparse.ArgumentParser('server')
parser.add_argument('-i', type = str, default = '127.0.0.1', help='frame')
parser.add_argument('-p', type = int, default = 1, help='frame')
flags = parser.parse_args()

BUFF_SIZE = 65536
UDP_IP = flags.i
UDP_PORT = flags.p
socket_address = (UDP_IP, UDP_PORT)
print(f"UDP target IP: {UDP_IP}")
print(f"UDP target port: {UDP_PORT}")

def main(win):
    ServerSocket = socket.socket(socket.AF_INET,     # Internet
                          socket.SOCK_DGRAM)  # UDP
    ServerSocket.bind(socket_address)

    ADDRESSES = []
    vert_imgs,hor_imgs = 2,2
    W,H = [(320,180),(320,240),(640,480),(480,360)][1]
    dim = (W,H)
    frame_out = np.zeros([H*vert_imgs,W*hor_imgs,3],dtype='uint8')

    ##############################################################################


    RUN = True
    locked_positions = {}
    grid = create_grid(locked_positions)
    clock = pygame.time.Clock() # create a timer
    vert_imgs,hor_imgs = layout[0],layout[1]

    change_piece = False
    current_piece = get_shape()
    next_piece = get_shape()

    fall_speed_real = FALL_SPEED
    fall_speed = fall_speed_real
    fall_speed_down = 0.3

    fall_time = 0
    level_time = 0
    score = 0
    frame_counter=0
    loc_h,loc_w = 0,0
    loc_h1,loc_w1,loc_h2,loc_w2 = 0,0,0,0
    left_wait = right_wait = rotate_wait = down_wait = 0
    ##############################################################################


    fps,st,frames_to_count,cnt = (0,0,20,0)
    ALLOW_CONTROL = False
    STATUS = b'inac'
    while RUN:
        # frame_out = np.zeros([H*vert_imgs,W*hor_imgs,3],dtype='uint8')
        ##############################################################################
        if 1:
            loc = functions.get_image_part(current_piece.x,current_piece.y,N_hor_blocks, N_ver_blocks, n_players_vert = vert_imgs, n_players_hor = hor_imgs)
            if len(loc) == 1:
                loc_h,loc_w = loc[0][0],loc[0][1]
            # elif len(loc) == 2:
            #     loc_h1,loc_w1 = loc[0][0],loc[0][1]
            #     loc_h2,loc_w2 = loc[1][0],loc[1][1]
            if loc_h < vert_imgs and loc_w < hor_imgs:
                player = cell[loc_h,loc_w]
            player_can_control = [player]
        ##############################################################################

        # RECEIVE MESSAGE FROM CLIENT
        packet, sender_addr = ServerSocket.recvfrom(BUFF_SIZE)
        if sender_addr not in ADDRESSES:
            ADDRESSES.append(sender_addr)
        if len(ADDRESSES) > 4:
            ADDRESSES = ADDRESSES[:4]

        ##############################################################################
        for user_number,addr in enumerate(ADDRESSES):
            if addr == sender_addr:
                break
        if user_number in player_can_control:
            STATUS = b'acti'
            command = packet[-4:]
        else:
            command = b'None'
            STATUS = b'inac'
        # print(command,user_number)
        ##############################################################################



        data = base64.b64decode(packet[:-4],' /')
        npdata = np.fromstring(data,dtype=np.uint8)
        frame_in = cv2.imdecode(npdata,1)
        frame_in = cv2.putText(frame_in,'FPS: '+str(fps)+' N='+str(user_number),(10,40),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)

        layoutt = [(0,0),(0,1),(1,0),(1,1)]
        x,y = layoutt[user_number]
        frame_out[x*H:(x+1)*H,y*W:(y+1)*W,:] = frame_in
    #     frame_in = cv2.putText(frame_in,'FPS: '+str(fps)+' N='+str(user_number),(10,40),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
        # frame_out = np.concatenate((frame_in,frame_in),0)
        # frame_out = np.concatenate((frame_out,frame_out),1)

       ##############################################################################
        if 1:
            # draw squazes over the image
            # create a block template
            block = np.ones([block_size,block_size,3])
            for h in range(len(grid)):
                for w in range(len(grid[h])):
                    if grid[h][w] != (0,0,0):
                        # fill colours
                        block[:,:,0] = grid[h][w][0]
                        block[:,:,1] = grid[h][w][1]
                        block[:,:,2] = grid[h][w][2]
                        frame_out[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size,:] = block

        # clock.tick(FPS)
        #############################################################################
        cv2.imshow("SERVER SIDE VIDEO",frame_out)

        # SEND MESSAGE TO CLIENT
        encoded,buffer = cv2.imencode('.jpg',frame_out,[cv2.IMWRITE_JPEG_QUALITY,25])
        message = base64.b64encode(buffer)
        ServerSocket.sendto(message+STATUS,sender_addr)


        ##############################################################################
        if 1:
            if command == b'left':
                left_wait += 1
            if command == b'righ':
                right_wait += 1
            if command == b'rota':
                rotate_wait += 1
            if command == b'down':
                down_wait += 1
        if 1:
            grid = create_grid(locked_positions)
            fall_time += clock.get_rawtime()
            level_time += clock.get_rawtime()
            clock.tick()

            # every 10 sec, shapes move 0.02 sec faster (peak at 0.25 sec)
            if level_time/1000 > 10:
                level_time = 0
                if fall_speed_real > 0.25:
                    fall_speed_real -= 0.02

            # if enough time (fall_speed) have passsed, piece moves down 1 block
            if fall_time/1000 > fall_speed:
                fall_time = 0
                current_piece.y += 1
                if not(valid_space(current_piece, grid)) and current_piece.y > 0:
                    current_piece.y -= 1
                    change_piece = True

            # CONTROLLING VIA GESTURES
            # quit the game if quit button has been pushed
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    RUN = False
                    cv2.destroyAllWindows()
            # "if you gesture to the LEFT for at least 4 frames, piece move LEFT"
            if left_wait >= 8:
                # if there is gestyre then decrease fall speed
                fall_speed = fall_speed_real
                current_piece.x -= 1
                if not (valid_space(current_piece, grid)):
                    current_piece.x += 1
                left_wait = right_wait = rotate_wait = down_wait = 0

            # "if you gesture to the RIGHT for at least 4 frames, piece move RIGHT"
            if right_wait >= 8:
                fall_speed = fall_speed_real
                current_piece.x += 1
                if not (valid_space(current_piece, grid)):
                    current_piece.x -= 1
                left_wait = right_wait = rotate_wait = down_wait = 0

            # "if you gesture to ROTATE  for at least 4 frames, piece ROTATES"
            if rotate_wait >= 8:
                fall_speed = fall_speed_real
                current_piece.rotation += 1
                if not (valid_space(current_piece, grid)):
                    current_piece.rotation -= 1
                left_wait = right_wait = rotate_wait = down_wait = 0

            # "if you gesture to go DOWN (no hand on the screen) for at least 5 frames, piece go DOWN (moves very fast)"
            if down_wait >= 5:
                # if there is no gestyre then increas fall speed
                fall_speed = fall_speed_real
                left_wait = right_wait = rotate_wait = down_wait = 0

            shape_pos = convert_shape_format(current_piece)

            for i in range(len(shape_pos)):
                x, y = shape_pos[i]
                if y > -1:
                    grid[y][x] = current_piece.color

            if change_piece:
                for pos in shape_pos:
                    p = (pos[0], pos[1])
                    locked_positions[p] = current_piece.color
                current_piece = next_piece
                next_piece = get_shape()
                change_piece = False
                fall_speed = fall_speed_real
                # down_wait = 0
                score += clear_rows(grid, locked_positions) * 10

            # update display
            pygame.display.update()

            if check_lost(locked_positions):
                win.fill((0,0,0))
                draw_text_middle(win, "YOU're INVALID!", 80, (255,255,255))
                pygame.display.update()
                pygame.time.delay(1500)
                RUN = False

    #         # PRINT INFO
    #         if frame_counter%15 == 0:
    #             print("FPS = ", str(int(clock.get_fps())))
    #             print("Current player = ",player)
    #             print("Score = ", score)
            frame_counter+=1
        ##############################################################################


        # for stable FPS
        key = cv2.waitKey(1) & 0xFF
        # to count FPS
        if cnt == frames_to_count:
            try:
                fps = round(frames_to_count/(time.time()-st))
                st=time.time()
                cnt=0
            except:
                pass
        cnt+=1
    
def main_menu(win):
    RUN = True
    while RUN:
        win.fill((0,0,0))
        draw_text_middle(win, 'Press Any Key To Play', 60, (255,255,255))
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                print('CLOSE THE GAME')
                win.fill((0,0,0))
                draw_text_middle(win, 'Press Any Key To Play', 60, (255,255,255))
                RUN = False
            if event.type == pygame.KEYDOWN:
                win.fill((0,0,0))
                draw_text_middle(win, 'You are playing', 60, (255,255,255))
                print('START THE GAME')
                main(win)
    pygame.display.quit()

# create a window
win = pygame.display.set_mode((s_width, s_height))
# run the main menu
main_menu(win)

Writing UDPs.py


# CLIENT

In [9]:
%%writefile UDPc.py

def main():
    import cv2, imutils, socket
    import numpy as np
    import base64
    import time

    import argparse
    parser = argparse.ArgumentParser('client')
    parser.add_argument('-i', type = str, default = '127.0.0.1', help='frame')
    parser.add_argument('-p', type = int, default = 1, help='frame')
    parser.add_argument('-f', type = int, default = 1, help='frame')
    parser.add_argument('-s', type = int, default = 1, help='frame')
    flags = parser.parse_args()

    import mediapipe as mp
    # Getting mediapipe: Hands ready
    mpHands = mp.solutions.hands
    hands = mpHands.Hands()
    mpDraw = mp.solutions.drawing_utils

    BUFF_SIZE = 65536
    UDP_IP = flags.i
    UDP_PORT = flags.p
    socket_address = (UDP_IP, UDP_PORT)
    print(f"UDP target IP: {UDP_IP}")
    print(f"UDP target port: {UDP_PORT}")

    if flags.s == 0:
        SOURCE = 0
    else:
        SOURCE = 'v'+str(flags.f)+'.mp4'

    ClientSocket = socket.socket(socket.AF_INET,     # Internet
                          socket.SOCK_DGRAM)  # UDP
    #  | socket.SOCK_NONBLOCK
    ClientSocket.setblocking(0)

    vid = cv2.VideoCapture(SOURCE)
    W,H = [(320,180),(320,240),(640,480),(480,360)][2]
    dim = (W,H)
    screen_coeff=1.5

    def movements(frame,left_wait,right_wait,rotate_wait,down_wait):
        # left_wait = right_wait = rotate_wait = down_wait = 0
        results = hands.process(frame)
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                for id, lm in enumerate(handLms.landmark):
                    h, w, c = frame.shape
                    if id == 0:
                        x = []
                        y = []
                    x.append(int((lm.x) * w))
                    y.append(int((1 - lm.y) * h))

                    # This will track the hand gestures
                    if len(y) > 20:
                        if (x[0] > x[3] > x[4]) and not(y[20] > y[17]):
                            left_wait += 1
                        if not(x[0] > x[3] > x[4]) and (y[20] > y[17]):
                            right_wait += 1
                        if (x[0] > x[3] > x[4]) and (y[20] > y[17]):
                            rotate_wait += 1
                mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS)
        else:
            down_wait += 1
        return frame,left_wait,right_wait,rotate_wait,down_wait

    def controlling(out,left_wait,right_wait,rotate_wait,down_wait):
        fall_speed = 0
        fall_speed_real = 0
        if 1:
            # "if you gesture to the LEFT for at least 4 frames, piece move LEFT"
            if left_wait >= 2:
                # if there is gestyre then decrease fall speed
                out = b'left'
                # fall_speed = fall_speed_real
                # current_piece.x -= 1
                # if not (valid_space(current_piece, grid)):
                #     current_piece.x += 1
                left_wait = right_wait = rotate_wait = down_wait = 0

            # "if you gesture to the RIGHT for at least 4 frames, piece move RIGHT"
            if right_wait >= 2:
                out = b'righ'
                # fall_speed = fall_speed_real
                # current_piece.x += 1
                # if not (valid_space(current_piece, grid)):
                #     current_piece.x -= 1
                left_wait = right_wait = rotate_wait = down_wait = 0

            # "if you gesture to ROTATE  for at least 4 frames, piece ROTATES"
            if rotate_wait >= 3:
                out = b'rota'
                # fall_speed = fall_speed_real
                # current_piece.rotation += 1
                # if not (valid_space(current_piece, grid)):
                #     current_piece.rotation -= 1
                left_wait = right_wait = rotate_wait = down_wait = 0

            # "if you gesture to go DOWN (no hand on the screen) for at least 5 frames, piece go DOWN (moves very fast)"
            if down_wait >= 3:
                out = b'down'
                # if there is no gestyre then increas fall speed
                # fall_speed = fall_speed_down
                left_wait = right_wait = rotate_wait = down_wait = 0
        return out,left_wait,right_wait,rotate_wait,down_wait

    left_wait = right_wait = rotate_wait = down_wait = 0
    out = b'None'
    zero_frame = np.zeros([H,W,3])
    success = False

    fps,st,frames_to_count,cnt = (0,0,20,0)
    STATUS = b'inac'
    while True:
        # READ A FRAME
        success,frame_out = vid.read() # read from camera
        if success:
            frame_out = cv2.flip(frame_out,1) # flip horizontally
        else:
            frame_out = zero_frame # return zero frame

        # RESIZE A FRAME
        frame_out = cv2.resize(frame_out, dim, interpolation = cv2.INTER_AREA)
        frame_out = imutils.resize(frame_out,width=320)

        # GESTURE RECKOGNITION
        if STATUS == b'acti':
            frame_out,left_wait,right_wait,rotate_wait,down_wait = movements(frame_out,left_wait,right_wait,rotate_wait,down_wait)
            out,left_wait,right_wait,rotate_wait,down_wait = controlling(out,left_wait,right_wait,rotate_wait,down_wait)
    #         frame_out = cv2.putText(frame_out,str(out),(50,100),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
            STATUS = b'inac'

        # ENCODE AND SEND A FRAME
        encoded,buffer = cv2.imencode('.jpg',frame_out,[cv2.IMWRITE_JPEG_QUALITY,25])
        message = base64.b64encode(buffer)
        ClientSocket.sendto(message+out,socket_address)

        # RECIEVE AND RESIZE A FRAME
        try:
            packet, sender_addr = ClientSocket.recvfrom(BUFF_SIZE) # recieve
            STATUS = packet[-4:]
            data = base64.b64decode(packet,' /') # decode
            npdata = np.fromstring(data,dtype=np.uint8) # decode
            frame_in = cv2.imdecode(npdata,1) # decode
            frame_in = cv2.putText(frame_in,'FPS_1: '+str(fps),(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2) # write fps for client
            # resize output image
            shape = frame_in.shape
            dim_show = (int(shape[1]*screen_coeff),int(shape[0]*screen_coeff))
            frame_in = cv2.resize(frame_in, dim_show, interpolation = cv2.INTER_AREA)
            # show on the screen
            cv2.imshow("CLIENT SIDE PROCESSED VIDEO",frame_in)
        except:
            None

        # for stable FPS
        key = cv2.waitKey(1) & 0xFF
        # to count FPS
        if cnt == frames_to_count:
            try:
                fps = round(frames_to_count/(time.time()-st))
                st=time.time()
                cnt=0
            except:
                pass    
        cnt+=1

if __name__ == "__main__":
    main()

Writing UDPc.py


In [10]:
%%writefile tetris_client.py
from UDPc import main

if __name__ == '__main__':
    main()

Writing tetris_client.py


# Dockerfile

In [11]:
%%writefile Dockerfile

FROM python:3.7.10

WORKDIR /usr/src/app

RUN apt-get update \
    && apt-get install -y python-opencv \
    && pip install numpy \
    && pip install opencv-python \
    && pip install mediapipe \
    && pip install imutils  

COPY UDPc.py ./

#ARG UDP_IP="192.168.223.24
#ARG UDP_PORT=5002


# only client
# !python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 1 -s 0
# CMD [ "python", "./UDPc.py", "-i" ,"192.168.223.24","-p","5002","-f" ,"1","-s","0"]
CMD [ "python", "./UDPc.py", "-i" ,"10.16.116.12","-p","5002","-f" ,"1","-s","0"]

Writing Dockerfile


In [12]:
!docker build -t udpc .

Sending build context to Docker daemon  981.5kB
Step 1/5 : FROM python:3.7.10
 ---> 3221bb99a8a9
Step 2/5 : WORKDIR /usr/src/app
 ---> Using cache
 ---> 2b40e0d0b7a8
Step 3/5 : RUN apt-get update     && apt-get install -y python-opencv     && pip install numpy     && pip install opencv-python     && pip install mediapipe     && pip install imutils
 ---> Using cache
 ---> 9a5ed0766faf
Step 4/5 : COPY UDPc.py ./
 ---> ee5c76aa0944
Step 5/5 : CMD [ "python", "./UDPc.py", "-i" ,"10.16.116.12","-p","5002","-f" ,"1","-s","0"]
 ---> Running in 8406259e972c
Removing intermediate container 8406259e972c
 ---> 214d45df770d
Successfully built 214d45df770d
Successfully tagged udpc:latest


# RUN

In [47]:
# UDP_IP = '192.168.66.62'      # M31
UDP_IP = '10.16.104.38'     # skoltech 5g
UDP_PORT = 5000

# only server
!python UDPs.py -i {UDP_IP} -p {UDP_PORT}

# only client (.py)
# !python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 1 -s 0
# only client (.exe)
# !.\tetris.exe -i {UDP_IP} -p {UDP_PORT} -f 1 -s 0

# server + 1 client (.py, camera)
# !python UDPs.py -i {UDP_IP} -p {UDP_PORT} & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 1 -s 0

# server + 1 client (video)
# !python UDPs.py -i {UDP_IP} -p {UDP_PORT} & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 7 -s 1

# server + 4 clients (video)
# !python UDPs.py -i {UDP_IP} -p {UDP_PORT} & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 1 & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 2 & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 3 & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 4 
# !python UDPs.py -i {UDP_IP} -p {UDP_PORT} & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 7 -s 1 & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 7 -s 1 & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 7 -s 1 & python UDPc.py -i {UDP_IP} -p {UDP_PORT} -f 7 -s 1

# run exe (windows)
# Desktop\tetris.exe -i 192.168.66.62 -p 5000 -f 1 -s 0
# Desktop\tetris.exe -i 10.16.104.38 -p 5000 -f 1 -s 0

# Docker
# !xhost +local:
# !docker run --device /dev/video0 --env="DISPLAY" --volume="/tmp/.X11-unix:/tmp/.X11-unix:rw" -p 5002:5002 -it udpc

UDPs.py:254: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  npdata = np.fromstring(data,dtype=np.uint8)


pygame 2.1.2 (SDL 2.0.18, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
UDP target IP: 10.16.104.38
UDP target port: 5000
START THE GAME
START THE GAME
START THE GAME
START THE GAME
START THE GAME
CLOSE THE GAME


# Create executable file

In [7]:
!pyinstaller tetris.py --add-data="C:\Anaconda\Lib\site-packages\mediapipe\modules;mediapipe/modules" -F -w

1454 INFO: PyInstaller: 4.10
1454 INFO: Python: 3.8.5 (conda)
1454 INFO: Platform: Windows-10-10.0.19041-SP0
1454 INFO: wrote C:\Users\kriku\WORKING1\tetris.spec
1457 INFO: UPX is not available.
1462 INFO: Extending PYTHONPATH with paths
['C:\\Users\\kriku\\WORKING1']
2063 INFO: checking Analysis
2064 INFO: Building Analysis because Analysis-00.toc is non existent
2064 INFO: Initializing module dependency graph...
2067 INFO: Caching module graph hooks...
2079 INFO: Analyzing base_library.zip ...
6390 INFO: Processing pre-find module path hook distutils from 'c:\\anaconda\\lib\\site-packages\\PyInstaller\\hooks\\pre_find_module_path\\hook-distutils.py'.
6390 INFO: distutils: retargeting to non-venv dir 'c:\\anaconda\\lib'
9633 INFO: Caching module dependency graph...
9831 INFO: running Analysis Analysis-00.toc
9848 INFO: Adding Microsoft.Windows.Common-Controls to dependent assemblies of final executable
  required by c:\anaconda\python.exe
10074 INFO: Analyzing C:\Users\kriku\WORKING1\